In [1]:
import json
from src.features.simple_feature_calculator import SimpleFeatureCalculator
from research.model_pick.candle_fetch import FusionCandles
from strategies.BinanceBtcDeapV1Voting.models.config import SSMContainer
from jesse.helpers import date_to_timestamp

from pathlib import Path
import lightgbm as lgb
import pandas as pd
import numpy as np

from research.plot import analyze_confidence_slices

for l in range(5, 8):
    for pn in range(1, 4):
        MODEL_TYPE = "r"
        LAG = l
        PRED_NEXT = pn
        THRESHOLD = 0
        LOWER_BOUND = -1
        UPPER_BOUND = 1
        MODEL_NAME = f"{MODEL_TYPE}_L{LAG}_N{PRED_NEXT}"

        STRATEGY_NAME = "BinanceBtcDemoBarV2"

        T_START = "2025-01-01"
        T_END = "2025-10-25"

        path_features = Path(".").parent / "strategies" / f"{STRATEGY_NAME}" / "models" / "feature_info.json"
        with open(path_features) as f:
            feature_info: dict[str, list[str]] = json.load(f)
        FEAT_FRACDIFF: list[str] = feature_info["fracdiff"]
        ALL_RAW_FEAT = []
        ALL_RAW_FEAT.extend(FEAT_FRACDIFF)
        ALL_RAW_FEAT.extend(feature_info[MODEL_NAME])
        ALL_RAW_FEAT = set(ALL_RAW_FEAT)
        ALL_RAW_FEAT = sorted(
            [
                i
                for i in ALL_RAW_FEAT
                if not i.startswith("deep_ssm") and not i.startswith("lg_ssm")
            ]
        )

        candle_container = FusionCandles(
            exchange="Binance Perpetual Futures", symbol="BTC-USDT", timeframe="1m"
        )
        candles = candle_container.get_candles(T_START, T_END)

        feat_calc = SimpleFeatureCalculator()
        feat_calc.load(candles, sequential=True)
        print("getting raw features...")
        df_raw_feats = pd.DataFrame.from_dict(feat_calc.get(ALL_RAW_FEAT))
        df_raw_feats.set_index(candles[:, 0].astype(int), inplace=True)
        print(f"{df_raw_feats.shape = }")

        model_deep_ssm = SSMContainer(model_type="deep_ssm")
        model_lg_ssm = SSMContainer(model_type="lg_ssm")

        df_deep_ssm = model_deep_ssm.transform(df_raw_feats[FEAT_FRACDIFF])
        df_lg_ssm = model_lg_ssm.transform(df_raw_feats[FEAT_FRACDIFF])
        df_feat_full = pd.concat([df_deep_ssm, df_lg_ssm, df_raw_feats], axis=1)

        test_features = df_feat_full[
            (df_feat_full.index.to_numpy() >= date_to_timestamp(T_START)) &
            (df_feat_full.index.to_numpy() < date_to_timestamp(T_END))
            ]

        with open(f"strategies/{STRATEGY_NAME}/models/feature_info.json", "r") as f:
            feat_info = json.load(f)
        df_selected_feats = test_features[feat_info[f"{MODEL_NAME}"]]

        timestamps = candles[:, 0].astype(np.int64)
        mask = (timestamps >= test_features.index[0]) & (timestamps <= test_features.index[-1])

        close_price = candles[:, 2][mask]
        datetimes = pd.to_datetime(test_features.index, unit="ms")
        assert len(datetimes) == len(close_price)

        model = lgb.Booster(model_file=f"strategies/{STRATEGY_NAME}/models/model_{MODEL_NAME}.txt")
        pred = model.predict(df_selected_feats)
        assert len(pred) == len(datetimes)

        slice_res = analyze_confidence_slices(
            datetimes,
            pred,
            close_price,
            PRED_NEXT,
            output_dir=f"./temp/{MODEL_TYPE}_L{LAG}_N{PRED_NEXT}",
            granularity=0.02,
            lower_bound=LOWER_BOUND,
            upper_bound=UPPER_BOUND,
            threshold=THRESHOLD,
        )

PyTorch configured: device=cpu, dtype=torch.float32


/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.12/site-packages/jesse/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


getting raw features...
df_raw_feats.shape = (3537, 3909)
Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.json
Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.json

⚠️ 警告：发现 328 个超出范围 [-1, 1] 的值
  - 占比: 9.27%
  - 实际范围: [-1.2324, 1.2264]
  - 超出范围的值将被归类到最近的边界切片


📊 时序对齐信息:
  - pred_next: 1 (预测未来第1根K线)
  - 原始数据长度: 3537
  - 对齐后数据长度: 3536
  - 丢弃的尾部数据: 1 条

已生成: slice_0.9800_1.0000_ratio_0.0161.jpg (切片区间: [0.9800, 1.0000] (做多), 样本占比: 1.61%)
已生成: slice_0.9600_0.9800_ratio_0.0181.jpg (切片区间: [0.9600, 0.9800] (做多), 样本占比: 1.81%)
已生成: slice_0.9400_0.9600_ratio_0.0187.jpg (切片区间: [0.9400, 0.9600] (做多), 样本占比: 1.87%)
已生成: slice_0.9200_0.9400_ratio_0.0181.jpg (切片区间: [0.9200, 0.9400] (做多), 样本占比: 1

In [47]:
from src.features.simple_feature_calculator import SimpleFeatureCalculator
from research.model_pick.candle_fetch import FusionCandles
from strategies.BinanceBtcDeapV1Voting.models.config import SSMContainer
from jesse.helpers import date_to_timestamp

from pathlib import Path
import lightgbm as lgb
import pandas as pd
import numpy as np

T_START = "2025-01-01"
T_END = "2025-10-25"

path_features = Path(".").parent / "strategies" / f"{STRATEGY_NAME}" / "models" / "feature_info.json"
with open(path_features) as f:
    feature_info: dict[str, list[str]] = json.load(f)
FEAT_FRACDIFF: list[str] = feature_info["fracdiff"]
ALL_RAW_FEAT = []
ALL_RAW_FEAT.extend(FEAT_FRACDIFF)
ALL_RAW_FEAT.extend(feature_info[MODEL_NAME])
ALL_RAW_FEAT = set(ALL_RAW_FEAT)
ALL_RAW_FEAT = sorted(
    [
        i
        for i in ALL_RAW_FEAT
        if not i.startswith("deep_ssm") and not i.startswith("lg_ssm")
    ]
)

candle_container = FusionCandles(
    exchange="Binance Perpetual Futures", symbol="BTC-USDT", timeframe="1m"
)
candles = candle_container.get_candles(T_START, T_END)

feat_calc = SimpleFeatureCalculator()
feat_calc.load(candles, sequential=True)
print("getting raw features...")
df_raw_feats = pd.DataFrame.from_dict(feat_calc.get(ALL_RAW_FEAT))
df_raw_feats.set_index(candles[:, 0].astype(int), inplace=True)
print(f"{df_raw_feats.shape = }")

model_deep_ssm = SSMContainer(model_type="deep_ssm")
model_lg_ssm = SSMContainer(model_type="lg_ssm")

df_deep_ssm = model_deep_ssm.transform(df_raw_feats[FEAT_FRACDIFF])
df_lg_ssm = model_lg_ssm.transform(df_raw_feats[FEAT_FRACDIFF])
df_feat_full = pd.concat([df_deep_ssm, df_lg_ssm, df_raw_feats], axis=1)

test_features = df_feat_full[
    (df_feat_full.index.to_numpy() >= date_to_timestamp(T_START)) &
    (df_feat_full.index.to_numpy() < date_to_timestamp(T_END))
]

getting raw features...
df_raw_feats.shape = (3537, 1732)
Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.json
Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.json


In [48]:
with open(f"strategies/{STRATEGY_NAME}/models/feature_info.json", "r") as f:
    feat_info = json.load(f)
df_selected_feats = test_features[feat_info[f"{MODEL_NAME}"]]

timestamps = candles[:, 0].astype(np.int64)
mask = (timestamps >= test_features.index[0]) & (timestamps <= test_features.index[-1])

close_price = candles[:, 2][mask]
datetimes = pd.to_datetime(test_features.index, unit="ms")
assert len(datetimes) == len(close_price)

model = lgb.Booster(model_file=f"strategies/{STRATEGY_NAME}/models/model_{MODEL_NAME}.txt")
pred = model.predict(df_selected_feats)
assert len(pred) == len(datetimes)

In [49]:
from research.plot import analyze_confidence_slices

slice_res = analyze_confidence_slices(
    datetimes,
    pred,
    close_price,
    PRED_NEXT,
    output_dir=f"./temp/{MODEL_TYPE}_L{LAG}_N{PRED_NEXT}",
    granularity=0.02,
    lower_bound=LOWER_BOUND,
    upper_bound=UPPER_BOUND,
    threshold=THRESHOLD,
)


⚠️ 警告：发现 320 个超出范围 [-1, 1] 的值
  - 占比: 9.05%
  - 实际范围: [-1.2967, 1.2075]
  - 超出范围的值将被归类到最近的边界切片


📊 时序对齐信息:
  - pred_next: 2 (预测未来第2根K线)
  - 原始数据长度: 3537
  - 对齐后数据长度: 3535
  - 丢弃的尾部数据: 2 条

已生成: slice_0.9800_1.0000_ratio_0.0107.jpg (切片区间: [0.9800, 1.0000] (做多), 样本占比: 1.07%)
已生成: slice_0.9600_0.9800_ratio_0.0164.jpg (切片区间: [0.9600, 0.9800] (做多), 样本占比: 1.64%)
已生成: slice_0.9400_0.9600_ratio_0.0170.jpg (切片区间: [0.9400, 0.9600] (做多), 样本占比: 1.70%)
已生成: slice_0.9200_0.9400_ratio_0.0161.jpg (切片区间: [0.9200, 0.9400] (做多), 样本占比: 1.61%)
已生成: slice_0.9000_0.9200_ratio_0.0124.jpg (切片区间: [0.9000, 0.9200] (做多), 样本占比: 1.24%)
已生成: slice_0.8800_0.9000_ratio_0.0139.jpg (切片区间: [0.8800, 0.9000] (做多), 样本占比: 1.39%)
已生成: slice_0.8600_0.8800_ratio_0.0133.jpg (切片区间: [0.8600, 0.8800] (做多), 样本占比: 1.33%)
已生成: slice_0.8400_0.8600_ratio_0.0102.jpg (切片区间: [0.8400, 0.8600] (做多), 样本占比: 1.02%)
已生成: slice_0.8200_0.8400_ratio_0.0105.jpg (切片区间: [0.8200, 0.8400] (做多), 样本占比: 1.05%)
已生成: slice_0.8000_0.8200_ratio_0.0088.jpg (切片

In [36]:
from strategies.BinanceBtcDeapV1Voting.models.config import LGBMContainer

mc = LGBMContainer(
    model_type=MODEL_TYPE,
    lag=LAG,
    pred_next=PRED_NEXT,
    threshold=THRESHOLD,
)
print(mc.MODEL_NAME)

c_L3_N2


In [32]:
# 根据图片设定合适的filters

GIVEUP_FILTERS = [
    (0, 0.02),
    (0.12, 0.14),
    (0.24, 0.26),
    (0.32, 0.38),
    (0.4, 0.42),
    (0.46, 0.48),
    (0.5, 0.52),
]

REVERSE_FILTERS = [
    (0.04, 0.08),
    (0.16, 0.18),
    (0.2, 0.24),
    (0.28, 0.3),
    (0.48, 0.5),
    (0.52, 0.54),
]

for f in GIVEUP_FILTERS:
    mc.add_giveup_filter(f[0], f[1])

for f in REVERSE_FILTERS:
    mc.add_reverse_filter(f[0], f[1])

In [34]:
mc.save_filters()

Filters saved to /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/model_c_L3_N1_filters.json
